# 第16章 类和函数


In [17]:
class Time:
    """Represents the time of day.
    
    attributes: hour, minute, second
    """

def print_time(t):
    """Prints a string representation of the time.
    
    t: Time object
    """
    print('%.2d:%.2d:%.2d' %(t.hour, t.minute, t.second))

def is_after(t1, t2):
    """比较两个时间点大小, 如果t1 晚于 t2, 则返回 True  t1比t2大,返回true
    
    挑战:不要用if语句。
    t1:第一个时间对象
    t2:第二个时间对象
    """
    # Python中，元组是逐个元素进行比较的，从第一个元素开始。如果第一个元素相同，则继续比较第二个元素，依此类推。
    return (t1.hour, t1.minute, t1.second) > (t2.hour, t2.minute, t2.second)


time = Time()
time.hour = 11
time.minute = 59
time.second = 30

time2 = Time()
time2.hour = 11
time2.minute = 59
time2.second = 31

print_time(time)
result = is_after(time, time2)
result

11:59:30


False

In [23]:
# 16.2 纯函数

# 下面章节中，我们将编写两个函数，实现时间计算。这两个函数展示了两种函数类型：纯函数以及修改器。
# 同时，也展示了一个我称之为 原型和补丁的开发范式，这是一种通过构建简单原型，逐步改进，从而解决复杂问题的方法。

# 以下为add_time的一个简单原型：
def add_time(t1, t2):
    sum = Time()
    sum.hour = t1.hour + t2.hour
    sum.minute = t1.minute + t2.minute
    sum.second = t1.second + t2.second
    return sum

# 该函数新建了一个 Time对象，并初始化其属性，同时返回对象的引用。这便称之为纯函数，因为它不会修改作为参数传入
# 的人和对象，同时也不会产生其他效果，比如显示值或提示用户输入，而仅仅是返回一个值。

# 问题在于，函数没有处理秒数或分钟数相加后超过60的情况。遇到这种情况，我们需要将每60秒进位到分钟，每60分钟进位到小时

# 优化后的版本：
def add_time(t1, t2):
    sum = Time()
    sum.hour = t1.hour + t2.hour
    sum.minute = t1.minute + t2.minute
    sum.second = t1.second + t2.second

    if sum.second >= 60:
        sum.second -= 60
        sum.minute += 1
    
    if sum.minute >= 60:
        sum.minute -= 60
        sum.hour += 1
    return sum
# 这个函数虽然正确，但是略显臃肿。稍后我们将看到一个简短版本。

start = Time()
start.hour = 9
start.minute = 45
start.second = 0
duration = Time()
duration.hour = 1
duration.minute = 35
duration.second = 0
done = add_time(start, duration)
print_time(done)   



11:20:00


In [43]:
# 16.3 修改器

# 有时，对于一个函数来说，直接修改其获取的参数对象，更为高效。在此情境中，更改对于调用者来说，是可见的。
# 这样的函数，被称之为修改器。

import copy

def increment(time, seconds):
    """
    创建并返回新的 Time对象，将seconds加入time时间里，考虑seconds远大于60的情况。
    time:时间对象
    seconds: 要增加的描述
    """
    time.second += seconds

    minutes = time.second // 60
    time.second = time.second % 60

    time.minute += minutes
    hour = time.minute // 60
    time.minute = time.minute % 60

    time.hour += hour

def increment(time, seconds):
    """
    编写一个纯函数 increment,使其创建并返回新的Time对象，而不是修改其参数
    """
    time = copy.copy(time)
    time.second += seconds

    minutes = time.second // 60
    time.second = time.second % 60

    time.minute += minutes
    hour = time.minute // 60
    time.minute = time.minute % 60

    time.hour += hour
    return time


print_time(time) 
new_time = increment(time, 5000)
print_time(new_time)  

14:05:20
15:28:40


**16.4 原型与规划**

刚刚展示的开发方案是“原型和补丁“。就是针对每个函数，编写一个可以完成基本运算的函数，然后对其测试，并逐步修正错误。

这种方法很有效，尤其是你尚未对问题有深入了解时。但是增量修正往往会产生大量过于复杂代码---因为需要眼珠特殊情况---和不可靠代码---因为你很难确定是否找到了所有异常。

另一种方法是采用设计开发，用这种方法，就是采用上帝视角处理问题，从而开发会容易很多。

second属性是“个位”，minute属性是“60位”，hour属性是“3600”位。

In [ ]:
# 这里给出另一种解决问题的方法---我们将Time对象转换为整数，然后利用计算机善于进行整数运算的优势。

def time_to_int(time):
    """将Time对象转为整数"""
    minutes = time.hour * 60 + time.minute
    seconds = minutes * 60 + time.seconds
    return seconds

def int_to_time(seconds):
    """将整数转换为Time
    divmod, 将第一个参数除以第二个参数，将商和余作为元组返回。
    """
    time = Time()
    # 这里得到一个元组，两个值，整除的数和余数
    minutes, time.seconds = divmod(seconds, 60)
    time.hour, time.mintue = divmod(minutes, 60)
    return time

# 你可能需要费点脑力，并运行一些测试，以说服自己这些函数是正确的。
# 一旦你确信他们正确，便可以用起重写add_time:

def add_time(t1, t2):
    seconds = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

# 这个版本相比原来的版本，更容易验证。试一下，用time_to_int和 int_to_time 来重写increment
def increment(time, seconds):
    seconds = time_to_int(time) + seconds
    return int_to_time(seconds)

如果我们注意到时间不过是60进制的数字，并预先编写转换函数（time_to_int 和 int_to_time），便可以得到一个更精简，易读，易调试，也更可靠的程序。

同时这也更利于后续添加新的功能。比如，假如要对两个 Time 对象相减，获取其时间间隔。直接的方法，便是通过借位实现。但是，使用转换函数，会更容易，也更可能正确。

讽刺的是，有时候将问题想得复杂（或更通用），反而会更容易解决（因为特殊情况会更少，出错概率也会更低）。

**16.5 调试**

如果 minute 和second 在 0 到 60 之间（包括0但不包括60），同时 hour是正数，那这个Time对象便是正确的。hour和minute应该是整数，但 second可以允许有小数部分。  

这种约束条件，叫做不变式，因为它们需要恒为真。换句话说，如果它们不是真，那肯定有某些地方出错了。

编写代码来检查不变式，可以帮助发现错误并找出原因。比如，你可能有个函数 valid_time，它会接收一个 Time 对象，如果违反了不变式的某个条件，返回 False；

In [ ]:
def valid_time(time):
    if time.hour < 0 or time.minute < 0 or time.second < 0:
        return False
    if time.minute >=60 or time.second >= 60:
        return False
    return True

# 在每个函数的开始部分，你都可以检查参数并确保它们合法：
def add_time(t1, t2):
    if not valid_time(t1) or not valid_time(t2):
        raise ValueError('invalid Time object in add_time')
    seconds = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

# 或者你也可以用 assert 语句，检查一个给定的不变式，如果失败，抛出异常
def add_time(t1, t2):
    assert valid_time(t1) and valid_time(t2)
    seconds = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

# assert 语句非常有用，因为它们区分了何为常规条件代码，何为检验错误的代码。

文涛：在此之前从没考虑过 assert 的用法，即使见到了也是唯恐避之不及，没有钻研一下它的用法。assert 断言，就是用来检验错误的，也能使代码变得更简洁和易懂。很好地区分了常规条件代码和检验错误的代码。

### 16.6 术语表

原型和补丁：一种通过先写程序初稿，然后测试，并修正发现的错误的开发方案。（现在自己最喜欢这种开发方案，不断修正发现的错误，并且感觉到对编程了解的更深入，也更熟练）

设计开发：一种开发方案，采用上帝视角处理问题，相比于增量开发或者原型开发，需要更多提前规划。（这就是我之前采用开发方式吗？总想多考虑一些问题，但因为经验不足，所以做的反而不够好。应该先从原型开发方案开始，逐步修正，这样的工作做得多了，就能更好的使用设计开发方案了）  

纯函数：一种不修改任何作为参数传入的对象的函数。多数纯函数有返回值。   

修改器：一种函数，会修改作为参数接收的一个或者多个对象。多数修改器都是没有返回值的；也就是说，它们返回None。（这个以前我用过很多次，就是传入两个对象，将第二个对象的值赋值给第一个，又或者做些处理计算。一开始还担心如果没有返回值会怎么样，会不会修改失败，使用的是旧数据。现在想，当时没分清楚纯函数和修改器的用法。只是将方法抽取了出来，其实也相当于在代码块中执行并修改了数据。）

函数式编程风格：一种程序设计风格，其中多数函数都是纯函数。

不变式：程序执行期间，应该始终为真的条件。（这个我经常用，判断一个参数是否正常。只是现在终于知道了它的术语名称，叫做不变式）

断言语句：检查条件是否满足，并在失败时抛出异常的语句。

### 16.7 习题集

**Exercise 16.1** 编写mul_time函数，接收一个Time对象以及一个数字，返回一个原始时间和数字的乘积的新时间对象。

然后用mul_time来编写一个函数，接收一个表示比赛结束时间的Time对象，以及一个表示距离的数字，返回一个表示平均配速（每英里耗时）的Time对象。

In [54]:


def time_to_int(time):
    """
    时间是60进制的,将所有时间转换成秒数.

    time:时间对象
    returns:返回转换后的秒数
    """
    minutes = time.hour * 60 + time.minute
    seconds = minutes * 60 + time.second
    return seconds

def int_to_time(seconds):
    """
    通过60进制, 将描述转换为时间对象. 写一个纯函数,创建一个新对象返回，

    seconds:全部的秒数
    returns: 由秒数创建好的时间对象.
    """
    time = Time()
    # 让我们再复习一遍，divmod会返回一个元组，整除的数和余数
    minutes, time.second = divmod(seconds, 60)
    time.hour, time.minute = divmod(minutes, 60)
    return time


def mul_time(time, num):
    """
    接收一个Time对象以及一个数字，返回一个原始时间和数字的乘积的新时间对象

    time:时间对象
    num:时间乘积倍数
    """
    seconds = time_to_int(time) * num
    return int_to_time(seconds)

# 编写一个函数，接收一个表示比赛结束时间的Time对象，以及一个表示距离的数字，返回一个表示平均配速的Time对象。
def divmod_time(time, num):
    seconds = time_to_int(time) / num
    return int_to_time(seconds)

time = Time()
time.hour = 2
time.minute = 50
time.second = 59

num = 26.2

# time1 = mul_time(time, num)
time1 = divmod_time(time, num)
print((time1.hour, time1.minute, time1.second))

(0.0, 6.0, 31.564885496183194)


In [1]:
from datetime import datetime, date

# 1. 使用 datetime模块编写程序，获取当前日期并打印星期几
# datetime.date 一个理想化的简单型日期。属性：year, month, and day.
def get_now_time():
    """获取当前日期并打印星期几"""
    time = date.today()
    weekday= time.isoweekday()
    print(f'当前日期是：{time}, 星期：{weekday}')
    return time

# 2. 编写个程序，输入生日，输出用户的年龄以及距离下个生日的天数，小时数，分钟数和秒数。
def days_until_birthday(birthday):
    """How long until my next birthday?"""
    today = datetime.today()
    # when is my birthday this year?
    next_birthday = datetime(today.year, birthday.month, birthday.day)

    # datetime对象之间的运算好像是可以直接用来比较和增减的，和数值一样。
    # 会不会是因为内部就和元组一样，能比较能运算。
    # if it has gone by, when will it be next yead
    if today > next_birthday:
        # 说明今年已经过了生日了
        next_birthday = datetime(today.year + 1, birthday.month, birthday.day)
    
    # subtraction on datetime objects returns a timedelta onject
    delta = next_birthday - today
    print(f'delta:{delta}')
    return delta.days

today = get_now_time()
birthday = datetime(2000, 10, 26)
days = days_until_birthday(birthday)
print(f'days:{days}')


当前日期是：2024-10-17, 星期：4
delta:8 days, 12:25:45.808771
days:8
